In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.baseline_functions as base

# restore saved variables
#%store -r summary_violent6_KY

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
data = data.drop(['arnold_fta_raw','arnold_nca_raw','arnold_nvca_raw'], axis=1)
x = data.loc[:,:'current_violence20']
y = data['violent_six_month'].values

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(X=x,
                                 Y=y,
                                 C=c,
                                 seed=816)
#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(X=x,
                           Y=y,
                           C=c,
                           seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(X=x,
                             Y=y, 
                             C=c,
                             seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [6,7]
rf_summary = base.RF(X=x,
                     Y=y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5]
n_estimators = [100,150]
xgb_summary = base.XGB(X=x,
                       Y=y, 
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators, 
                       seed=816)

#### save results
summary_violent6_KY = {"Logistic": logistic_summary,
                       "Lasso": lasso_summary,
                       "LinearSVM": svm_summary,
                       "RF": rf_summary,
                       "XGBoost": xgb_summary}
%store summary_violent6_KY

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:84: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:84: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:31: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv = tp / (tp + fp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:44: RuntimeWarning: divide by zero encountered in longlong_scalars
  ratio = fn / fp
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:84: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Re

Stored 'summary_violent6_KY' (dict)


In [4]:
results = []
auc = [] ## for table writing -- the performance range 
for model_name, model_summary in summary_violent6_KY.items():
    results.append([model_name, np.mean(model_summary['holdout_test_auc']), np.mean(model_summary['auc_diffs'])])
    auc.append(np.mean(model_summary['holdout_test_auc']))
results

[['Logistic', 0.8670572647537774, 0.0015522771421075277],
 ['Lasso', 0.8670062880382534, 0.0013482496076741234],
 ['LinearSVM', 0.8656059282923471, 0.001441406767736031],
 ['RF', 0.8757934872018233, 0.01164376281616919],
 ['XGBoost', 0.881483876466165, 0.014446159624958743]]

In [5]:
params = [[model_name, model_summary['best_param']] for model_name, model_summary in summary_violent6_KY.items()]
params

[['Logistic',
  [{'C': 0.01}, {'C': 0.01}, {'C': 0.01}, {'C': 0.01}, {'C': 0.01}]],
 ['Lasso', [{'C': 0.01}, {'C': 0.1}, {'C': 0.01}, {'C': 0.01}, {'C': 0.01}]],
 ['LinearSVM', [{'C': 1}, {'C': 1}, {'C': 1}, {'C': 1}, {'C': 1}]],
 ['RF',
  [{'max_depth': 7, 'n_estimators': 150},
   {'max_depth': 7, 'n_estimators': 200},
   {'max_depth': 7, 'n_estimators': 150},
   {'max_depth': 7, 'n_estimators': 200},
   {'max_depth': 7, 'n_estimators': 200}]],
 ['XGBoost',
  [{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150}]]]

In [6]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\Baselines\\Six Month\\"
results = [["Violent", np.str((round(np.mean(logistic_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(logistic_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(lasso_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(lasso_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(svm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(svm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(rf_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(rf_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(xgb_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(xgb_summary['holdout_test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3)]]
with open(path + 'Six Month Baseline Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)